In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers
from keras.utils import Sequence
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from PIL import Image
import time
#from tensorflow.keras.layers.experimental.preprocessing import ToTensor
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler

In [ ]:
import numpy as np
train, val = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

print("Train data is: ", train[:5], "\n\n", "Length of train data is: ", len(train), "\n")
print("Train data is: ", dev[:5], "\n\n", "Length of train data is: ", len(dev), "\n")




from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=42)


#pip install splitfolders
import splitfolders

image_directory= r'C:\Users\ugoch\Desktop\16 OBU students_Model\CNN from scratch\data_mv'
splitfolders.ratio(image_directory, output="output",
        seed=42, ratio=(0.7, 0.15, 0.15), group_prefix=None, move=False) # default values


In [ ]:
train_red_dir='/content/drive/MyDrive/DATA_SET_1/Train_/Train_rgb_nir/train_red'
train_green_dir='/content/drive/MyDrive/DATA_SET_1/Train_/Train_rgb_nir/train_green'
train_blue_dir='/content/drive/MyDrive/DATA_SET_1/Train_/Train_rgb_nir/train_blue'
train_nir_dir='/content/drive/MyDrive/DATA_SET_1/Train_/Train_rgb_nir/train_nir'
train_gt_dir='/content/drive/MyDrive/DATA_SET_1/Train_/Train_rgb_nir/train_gt'

val_red_dir ='/content/drive/MyDrive/DATA_SET_1/Train_/Val_rgb_nir/val_red'
val_green_dir = '/content/drive/MyDrive/DATA_SET_1/Train_/Val_rgb_nir/val_green'
val_blue_dir = '/content/drive/MyDrive/DATA_SET_1/Train_/Val_rgb_nir/val_blue'
val_nir_dir = '/content/drive/MyDrive/DATA_SET_1/Train_/Val_rgb_nir/val_nir'
val_gt_dir = '/content/drive/MyDrive/DATA_SET_1/Train_/Val_gt'

test_red_dir ='/content/drive/MyDrive/DATA_SET_1/Test_/Test_rgb_nir/test_red'
test_green_dir = '/content/drive/MyDrive/DATA_SET_1/Test_/Test_rgb_nir/test_green'
test_blue_dir = '/content/drive/MyDrive/DATA_SET_1/Test_/Test_rgb_nir/test_blue'
test_nir_dir = '/content/drive/MyDrive/DATA_SET_1/Test_/Test_rgb_nir/test_nir'

user_batch_size = 1

In [ ]:
class TrainDataGenerator(Sequence):
    def __init__(self, base_path, red_dir, green_dir, blue_dir, nir_dir, gt_dir, batch_size):
        self.base_path = base_path
        self.batch_size = batch_size
        self.red_dir = red_dir
        self.green_dir =  green_dir
        self.blue_dir =  blue_dir
        self.nir_dir = nir_dir
        self.gt_dir = gt_dir
        
        self.file_names = os.listdir(self.red_dir)
        self.file_names = [ '_'.join(i.split('_')[1:]) for i in self.file_names]
        self.num_samples = len(self.file_names)

    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    def preprocess_image(self, red_img, green_img, blue_img, nir_img, gt_img):
        transforms = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
            #tf.keras.layers.experimental.preprocessing.ToTensor()
        ])

        red = transforms(red_img)
        green = transforms(green_img)
        blue = transforms(blue_img)
        nir = transforms(nir_img)
        gt = transforms(gt_img)

        return red, green, blue, nir, gt

    def __getitem__(self, idx):
        batch_files = self.file_names[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = []
        batch_y = []
        for file_name in batch_files:
            #print(self.red_dir / 'red_' + file_name)
            red_img = np.array(Image.open(os.path.join(self.red_dir, 'red_' + file_name)))
            green_img = np.array(Image.open(os.path.join(self.green_dir, 'green_' + file_name)))
            blue_img = np.array(Image.open(os.path.join(self.blue_dir, 'blue_' + file_name)))
            nir_img = np.array(Image.open(os.path.join(self.nir_dir, 'nir_' + file_name)))
            gt_img = np.array(Image.open(os.path.join(self.gt_dir, 'gt_' + file_name)))

            #Preprocessing
            red, green, blue, nir, gt = self.preprocess_image(red_img, green_img, blue_img, nir_img, gt_img)
            x = np.stack([red, green, blue, nir], axis=-1)
            y = gt
            batch_x.append(x)
            batch_y.append(y)
       
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y


In [ ]:
train_data = TrainDataGenerator(base_path='/content/drive/MyDrive/DATA_SET_1/Train_/Train_rgb_nir', red_dir = train_red_dir, green_dir = train_green_dir, blue_dir = train_blue_dir, nir_dir = train_nir_dir, gt_dir = train_gt_dir, batch_size=user_batch_size)
val_data = TrainDataGenerator(base_path='/content/drive/MyDrive/DATA_SET_1/Train_/Val_rgb_nir', red_dir = val_red_dir, green_dir = val_green_dir, blue_dir = val_blue_dir, nir_dir = val_nir_dir, gt_dir = val_gt_dir, batch_size=user_batch_size)

print(len(train_data))
print(len(val_data))
print(train_data.num_samples)
print(val_data.num_samples)


6000
2400
6000
2400


In [ ]:
class TestDataGenerator(Sequence):
    def __init__(self, base_path, red_dir, green_dir, blue_dir, nir_dir, batch_size):
        self.base_path = base_path
        self.batch_size = batch_size
        self.red_dir = red_dir
        self.green_dir =  green_dir
        self.blue_dir =  blue_dir
        self.nir_dir = nir_dir
        
        self.file_names = os.listdir(self.red_dir)
        self.file_names = [ '_'.join(i.split('_')[1:]) for i in self.file_names]
        self.num_samples = len(self.file_names)

    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    def preprocess_image(self, red_img, green_img, blue_img, nir_img, gt_img):
        transforms = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
            #tf.keras.layers.experimental.preprocessing.ToTensor()
        ])

        red = transforms(red_img)
        green = transforms(green_img)
        blue = transforms(blue_img)
        nir = transforms(nir_img)

        return red, green, blue, nir

    def __getitem__(self, idx):
        batch_files = self.file_names[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = []
        batch_y = []
        for file_name in batch_files:
            red_img = np.array(Image.open(self.red_dir / 'red_' + file_name))
            green_img = np.array(Image.open(self.green_dir / 'green_' + file_name))
            blue_img = np.array(Image.open(self.blue_dir / 'blue_' + file_name))
            nir_img = np.array(Image.open(self.nir_dir / 'nir_' + file_name))
            gt_img = np.array(Image.open(self.gt_dir / 'gt_' + file_name))

            #Preprocessing
            red, green, blue, nir, gt = self.preprocess_image(red_img, green_img, blue_img, nir_img, gt_img)
            x = np.stack([red, green, blue, nir], axis=-1)
            y = gt
            batch_x.append(x)
            batch_y.append(y)
       
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y


In [ ]:
test_data =  TestDataGenerator(base_path='/content/drive/MyDrive/DATA_SET_1/Train_/Test_rgb_nir', red_dir = test_red_dir, green_dir = test_green_dir, blue_dir = test_blue_dir, nir_dir = test_nir_dir, batch_size=user_batch_size)
print(len(test_data))
print(test_data.num_samples)

9201
9201


In [ ]:
def double_conv(inputs, filters, kernel_size=3, padding='same', activation='relu'):
    conv = Conv2D(filters, kernel_size, padding=padding, activation=activation)(inputs)
    conv = Conv2D(filters, kernel_size, padding=padding, activation=activation)(conv)
    return conv

def UNET(in_channels, out_channels):
    inputs = tf.keras.layers.Input(shape=(None, None, in_channels))

    # Contracting path
    conv1 = double_conv(inputs, 64)
    pool1 = MaxPooling2D(pool_size=2)(conv1)
    conv2 = double_conv(pool1, 128)
    pool2 = MaxPooling2D(pool_size=2)(conv2)
    conv3 = double_conv(pool2, 256)
    pool3 = MaxPooling2D(pool_size=2)(conv3)
    conv4 = double_conv(pool3, 512)
    pool4 = MaxPooling2D(pool_size=2)(conv4)
    conv5 = double_conv(pool4, 1024)

    # Expanding path
    upconv4 = UpSampling2D(size=2)(conv5)
    concat4 = Concatenate()([conv4, upconv4])
    upconv3 = UpSampling2D(size=2)(concat4)
    concat3 = Concatenate()([conv3, upconv3])
    upconv2 = UpSampling2D(size=2)(concat3)
    concat2 = Concatenate()([conv2, upconv2])
    upconv1 = UpSampling2D(size=2)(concat2)
    concat1 = Concatenate()([conv1, upconv1])
    outputs = Conv2D(out_channels, kernel_size=1, padding='same', activation='sigmoid')(concat1)

    model = Model(inputs=inputs, outputs=outputs)
    return model



def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

model = UNET(4, 1)
model.summary()
model.compile(optimizer='adam', loss = [jacard_coef_loss], metrics=[jacard_coef])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 4)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   2368        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, None, None,   36928       ['conv2d[0][0]']                 
                                64)                                                           

In [ ]:
#Callbacks
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

def getCallbacks(numEpochs):
    exponential_decay_fn = exponential_decay(0.0001, numEpochs)
    
    lr_scheduler = LearningRateScheduler(
        exponential_decay_fn, #function
        verbose=1
    )
    
    checkpoint = ModelCheckpoint(
        filepath = os.path.join('{}.h5'.format('UNET')),
        save_best_only = True,
    #     save_weights_only = False,
        monitor = 'val_loss',
        mode = 'auto',
        verbose = 1
    )
    
    earlystop = EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0.001,
        patience = 6,
        mode = 'auto',
        verbose = 1,
        restore_best_weights = True
    )
  
    callbacks = [checkpoint, earlystop, lr_scheduler]

    return callbacks

In [ ]:
 numEpochs = 10
 history = model.fit(
        train_data,
        # steps_per_epoch=np.ceil(len(os.listdir(dir_struct_dict['train_images']))/user_batch_size),
        validation_data = val_data,
        batch_size=user_batch_size,
        # validation_steps = np.ceil(len(os.listdir(dir_struct_dict['val_images']))/user_batch_size),
        epochs = numEpochs,
        callbacks=getCallbacks(numEpochs),
        use_multiprocessing=False,
        verbose=1
    )

FileNotFoundError: ignored

In [ ]:
"""# Parameters
params = {'dim': (32,32,32),
          'batch_size': 32,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

# Datasets
partition = # IDs
labels = # Labels

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = Sequential()
[...] # Architecture
model.compile()

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)


Model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)"""